In [8]:
import numpy as np
import pandas as pd
import openpyxl 
import os
%run ccp_func.ipynb

In [9]:
def nasdaq_colect(quarter):
    
    #quarter = format: 'Q3-2017'
    
    ccp = 'Nasdaq'
    reverse = quarter.split('-')
    reverse = reverse[1] + '_' + reverse[0]
    
    #basic
    b_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/' + ccp.capitalize() + '-' + 
                           quarter.lower() + '.xlsx', sheet_name='AggregatedDataFile')
    #files in use
    a_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/' + ccp.capitalize() + '-' + 
                           quarter.lower() + '.xlsx', 
                           sheet_name=ccp.capitalize() + '_DataFile_4_4a_' + reverse.upper())
    c_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/' + ccp.capitalize() + '-' + 
                           quarter.lower() + '.xlsx', 
                           sheet_name=ccp.capitalize() + '_DataFile_6_1_' + reverse.upper())
    d_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/' + ccp.capitalize() + '-' + 
                           quarter.lower() + '.xlsx', 
                           sheet_name=ccp.capitalize() + '_DataFile_6.2_' + reverse.upper())
    e_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/' + ccp.capitalize() + '-' + 
                           quarter.lower() + '.xlsx', 
                           sheet_name=ccp.capitalize() + '_DataFile_18_2_' + reverse.upper())
    g_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/' + ccp.capitalize() + '-' + 
                           quarter.lower() + '.xlsx', 
                           sheet_name=ccp.capitalize() + '_DataFile_23_' + reverse.upper())
    ex_rate = pd.read_excel('../CCP/data/kursy-walutowe-CCP.xlsx', sheet_name='Arkusz1')

    #extra col 'ex_rate' in g_file for counting '23.1.2'
    add_ex_rate(g_file, ex_rate, quarter)

    #recalculate g_file to EUR (so recalculations for '23.1.2' and '23.2.1' in f_file are skipped)
    g_file['23.1.2'] = g_file['23.1.2'] // g_file['ex_rate']
    g_file['23.2.1'] = g_file['23.2.1'] // g_file['ex_rate']

    #final file
    f_file = pd.DataFrame({'CCP':ccp.capitalize()}, index=[0,1,2])
    f_file['quarter'] = quarter
    f_file['clearning service'] = b_file['ReportLevelIdentifier']
    f_file['currency'] = b_file['Currency']
    f_file['4.1.1'] = b_file['4.1.1']
    f_file['4.1.2'] = b_file['4.1.2']
    f_file['4.1.3'] = b_file['4.1.3']
    f_file['4.1.4'] = b_file['4.1.4']
    f_file['4.1.5'] = b_file['4.1.5']
    f_file['4.1.6'] = b_file['4.1.6']
    f_file['4.1.7'] = b_file['4.1.7']
    f_file['4.1.8'] = b_file['4.1.8']
    f_file['4.1.9'] = b_file['4.1.9']
    f_file['4.1.10'] = b_file['4.1.10']

    #extra col in f_file for exchange rate
    for row in range(len(f_file)):
        if f_file.loc[row, 'currency'] == 'SEK':
            f_file.loc[row, 'ex_rate'] = ex_rate[ex_rate['Exchange rates'].isin(['EUR/SEK'])][quarter].values
        elif f_file.loc[row, 'currency'] == 'NOK':
            f_file.loc[row, 'ex_rate'] = ex_rate[ex_rate['Exchange rates'].isin(['EUR/NOK'])][quarter].values
        elif f_file.loc[row, 'currency'] == 'EUR':
            f_file.loc[row, 'ex_rate'] = 1
        else:
            pass

    # 4.4.7.PDA & 4.4.7.MA
    f_file['4.4.7.PDA'] = a_file[a_file.Description == 'PeakDayAmountInPast12Months'].reset_index()['4.4.7']
    f_file['4.4.7.MA'] = a_file[a_file.Description == 'MeanAverageOverPrevious12Months'].reset_index()['4.4.7']

    #6.1.1.CLIENT = 'Client_Net' + 'Client_Gross'
    for row in range(len(f_file)):
        if f_file.loc[row, 'clearning service'] == 'Financial Markets':
            f_file.loc[row, '6.1.1.CLIENT'] = sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & c_file.ReportLevelIdentifier.isin(['Financial Markets'])]['6.1.1'])
        elif f_file.loc[row, 'clearning service'] == 'Commodities':
            f_file.loc[row, '6.1.1.CLIENT'] = sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & c_file.ReportLevelIdentifier.isin(['Commodities'])]['6.1.1'])
        elif f_file.loc[row, 'clearning service'] == 'Seafood':
            f_file.loc[row, '6.1.1.CLIENT'] = sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & c_file.ReportLevelIdentifier.isin(['Seafood'])]['6.1.1'])
        else:
            pass


    #6.1.1.TOTAL = 'House_Net' + 'Client_Net' + 'Client_Gross'    
    for row in range(len(f_file)):
        if f_file.loc[row, 'clearning service'] == 'Financial Markets':
            f_file.loc[row, '6.1.1.TOTAL'] = sum(c_file[c_file.Description.isin(['House_Net', 'Client_Net', 'Client_Gross']) & c_file.ReportLevelIdentifier.isin(['Financial Markets'])]['6.1.1'])
        elif f_file.loc[row, 'clearning service'] == 'Commodities':
            f_file.loc[row, '6.1.1.TOTAL'] = sum(c_file[c_file.Description.isin(['House_Net', 'Client_Net', 'Client_Gross']) & c_file.ReportLevelIdentifier.isin(['Commodities'])]['6.1.1'])
        elif f_file.loc[row, 'clearning service'] == 'Seafood':
            f_file.loc[row, '6.1.1.TOTAL'] = sum(c_file[c_file.Description.isin(['House_Net', 'Client_Net', 'Client_Gross']) & c_file.ReportLevelIdentifier.isin(['Seafood'])]['6.1.1'])
        else:
            pass

    #6.2.15.PRE & POST
    f_file['6.2.15.PRE'] = d_file[d_file.Description == 'TotalIM_PreHaircut'].reset_index()['6.2.15']
    f_file['6.2.15.POST'] = d_file[d_file.Description == 'TotalIM_PostHaircut'].reset_index()['6.2.15']

    #number of CCP
    for row in range(len(f_file)):
        try:
            if f_file.loc[row, 'clearning service'] == 'Financial Markets':
                f_file.loc[row, '18.1.1.1'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                       (b_file.ReportLevelIdentifier == 'Financial Markets')]['18.1.1.1'].values
            elif f_file.loc[row, 'clearning service'] == 'Commodities':
                f_file.loc[row, '18.1.1.1'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                       (b_file.ReportLevelIdentifier == 'Commodities')]['18.1.1.1'].values
            elif f_file.loc[row, 'clearning service'] == 'Seafood':
                f_file.loc[row, '18.1.1.1'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                       (b_file.ReportLevelIdentifier == 'Seafood')]['18.1.1.1'].values
        except:
            pass

    for row in range(len(f_file)):
        try:
            if f_file.loc[row, 'clearning service'] == 'Financial Markets':
                f_file.loc[row, '18.1.1.2'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                       (b_file.ReportLevelIdentifier == 'Financial Markets')]['18.1.1.2'].values
            elif f_file.loc[row, 'clearning service'] == 'Commodities':
                f_file.loc[row, '18.1.1.2'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                       (b_file.ReportLevelIdentifier == 'Commodities')]['18.1.1.2'].values
            elif f_file.loc[row, 'clearning service'] == 'Seafood':
                f_file.loc[row, '18.1.1.2'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                       (b_file.ReportLevelIdentifier == 'Seafood')]['18.1.1.2'].values
        except:
            pass

    #number od CCP participants (average)
    for row in range(len(f_file)):
        try:
            if f_file.loc[row, 'clearning service'] == 'Financial Markets':
                f_file.loc[row, '18.1.2.2'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                       (b_file.ReportLevelIdentifier == 'Financial Markets')]['18.1.2.2'].values
            elif f_file.loc[row, 'clearning service'] == 'Commodities':
                f_file.loc[row, '18.1.2.2'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                       (b_file.ReportLevelIdentifier == 'Commodities')]['18.1.2.2'].values
            elif f_file.loc[row, 'clearning service'] == 'Seafood':
                f_file.loc[row, '18.1.2.2'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                       (b_file.ReportLevelIdentifier == 'Seafood')]['18.1.2.2'].values
        except:
            pass

    #number of clients
    f_file['19.1.1'] = b_file['19.1.1']
    f_file['19.1.2'] = b_file['19.1.2']

    #23.1.2.OTC and 23.1.2.ETD
    for row in range(len(f_file)):
        if f_file.loc[row, 'clearning service'] == 'Financial Markets':
            f_file.loc[row, '23.1.2.OTC'] = sum(g_file[(g_file.Description.map(lambda x: 'OTC' in x) == True) & 
                                                   (g_file.ReportLevelIdentifier == 'Financial Markets')]['23.1.2'])
        elif f_file.loc[row, 'clearning service'] == 'Commodities':
            f_file.loc[row, '23.1.2.OTC'] = sum(g_file[(g_file.Description.map(lambda x: 'OTC' in x) == True) & 
                                                   (g_file.ReportLevelIdentifier == 'Commodities')]['23.1.2'])
        elif f_file.loc[row, 'clearning service'] == 'Seafood':
            f_file.loc[row, '23.1.2.OTC'] = sum(g_file[(g_file.Description.map(lambda x: 'OTC' in x) == True) & 
                                                   (g_file.ReportLevelIdentifier == 'Seafood')]['23.1.2'])
        else:
            pass

    for row in range(len(f_file)):
        if f_file.loc[row, 'clearning service'] == 'Financial Markets':
            f_file.loc[row, '23.1.2.ETD'] = sum(g_file[(g_file.Description.map(lambda x: 'ETD' in x) == True) & 
                                                   (g_file.ReportLevelIdentifier == 'Financial Markets')]['23.1.2'])
        elif f_file.loc[row, 'clearning service'] == 'Commodities':
            f_file.loc[row, '23.1.2.ETD'] = sum(g_file[(g_file.Description.map(lambda x: 'ETD' in x) == True) & 
                                                   (g_file.ReportLevelIdentifier == 'Commodities')]['23.1.2'])
        elif f_file.loc[row, 'clearning service'] == 'Seafood':
            f_file.loc[row, '23.1.2.ETD'] = sum(g_file[(g_file.Description.map(lambda x: 'ETD' in x) == True) & 
                                                   (g_file.ReportLevelIdentifier == 'Seafood')]['23.1.2'])
        else:
            pass

    #23.2.1.OTC and 23.2.1.ETD
    for row in range(len(f_file)):
        if f_file.loc[row, 'clearning service'] == 'Financial Markets':
            f_file.loc[row, '23.2.1.OTC'] = sum(g_file[(g_file.Description.map(lambda x: 'OTC' in x) == True) & 
                                                   (g_file.ReportLevelIdentifier == 'Financial Markets')]['23.2.1'])
        elif f_file.loc[row, 'clearning service'] == 'Commodities':
            f_file.loc[row, '23.2.1.OTC'] = sum(g_file[(g_file.Description.map(lambda x: 'OTC' in x) == True) & 
                                                   (g_file.ReportLevelIdentifier == 'Commodities')]['23.2.1'])
        elif f_file.loc[row, 'clearning service'] == 'Seafood':
            f_file.loc[row, '23.2.1.OTC'] = sum(g_file[(g_file.Description.map(lambda x: 'OTC' in x) == True) & 
                                                   (g_file.ReportLevelIdentifier == 'Seafood')]['23.2.1'])
        else:
            pass

    for row in range(len(f_file)):
        if f_file.loc[row, 'clearning service'] == 'Financial Markets':
            f_file.loc[row, '23.2.1.ETD'] = sum(g_file[(g_file.Description.map(lambda x: 'ETD' in x) == True) & 
                                                   (g_file.ReportLevelIdentifier == 'Financial Markets')]['23.2.1'])
        elif f_file.loc[row, 'clearning service'] == 'Commodities':
            f_file.loc[row, '23.2.1.ETD'] = sum(g_file[(g_file.Description.map(lambda x: 'ETD' in x) == True) & 
                                                   (g_file.ReportLevelIdentifier == 'Commodities')]['23.2.1'])
        elif f_file.loc[row, 'clearning service'] == 'Seafood':
            f_file.loc[row, '23.2.1.ETD'] = sum(g_file[(g_file.Description.map(lambda x: 'ETD' in x) == True) & 
                                                   (g_file.ReportLevelIdentifier == 'Seafood')]['23.2.1'])
        else:
            pass

    #convert values to EUR for variable from '4.1.1' to '4.1.10' and 
    #'4.4.7.PDA', '4.4.7.MA', '6.6.1.TOTAL', '6.6.1.CLIENT'
    f_file['4.1.1'] = f_file['4.1.1'] // f_file['ex_rate']
    f_file['4.1.2'] = f_file['4.1.2'] // f_file['ex_rate']
    f_file['4.1.3'] = f_file['4.1.3'] // f_file['ex_rate']
    f_file['4.1.4'] = f_file['4.1.4'] // f_file['ex_rate']
    f_file['4.1.5'] = f_file['4.1.5'] // f_file['ex_rate']
    f_file['4.1.6'] = f_file['4.1.6'] // f_file['ex_rate']
    f_file['4.1.7'] = f_file['4.1.7'] // f_file['ex_rate']
    f_file['4.1.8'] = f_file['4.1.8'] // f_file['ex_rate']
    f_file['4.1.9'] = f_file['4.1.9'] // f_file['ex_rate']
    f_file['4.1.10'] = f_file['4.1.10'] // f_file['ex_rate']
    f_file['4.4.7.PDA'] = f_file['4.4.7.PDA'] // f_file['ex_rate']
    f_file['4.4.7.MA'] = f_file['4.4.7.MA'] // f_file['ex_rate']
    f_file['6.1.1.TOTAL'] = f_file['6.1.1.TOTAL'] // f_file['ex_rate']
    f_file['6.1.1.CLIENT'] = f_file['6.1.1.CLIENT'] // f_file['ex_rate']
    f_file['6.2.15.PRE'] = f_file['6.2.15.PRE'] // f_file['ex_rate']
    f_file['6.2.15.POST'] = f_file['6.2.15.POST'] // f_file['ex_rate']
    f_file['currency'] = 'EUR'

    #drop 'ex_rate'
    f_file = f_file.drop(columns=['ex_rate'])

    writer = pd.ExcelWriter("data/" + ccp.lower() + '/' + ccp.lower() + '-' + quarter.upper() + '.xlsx')
    f_file.to_excel(writer, ccp.lower() + '-' + quarter.upper())
    writer.save()

In [10]:
nasdaq_colect('Q1-2018')